# Find all the types of documents posible in supercias
### We need to define how many types documents and their frequency in supercias

In [2]:
packages = ['pymongo', 'gridfs', 'pandas']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        !pip install {package}

import os
from pymongo import MongoClient
import pandas as pd
from functions.pdf_operations import PDFOperations
endpoint = '10.0.10.5:27017' # use vpn
#endpoint = '192.168.1.10:27017'
#endpoint = '192.168.229.55:27017'
database = 'supercias'
collection = 'companies'
# Connect to MongoDB
db = MongoClient('mongodb://'+endpoint)[database]
# get collection
collection = db[collection]
# where the pdfs will be stored 
pdfs_path = '../storage/pdfs/'
# pdf operator
pdf_op = PDFOperations(db, 'companies', pdfs_path)
#find pdf file
companies_cursor = collection.find()

companies


Let seperate the docuemt by the type they can belong to: Generales, Economicos  and Judiciales

In [20]:
# now lets get all the documents for every company
companies = collection.find()
Documentos_generales = []
Documentos_economicos = []
Documentos_juridicos = []

for company in companies:
    # get the Documentos online
    try:
        documentos = company['Documentos online']['downloaded']
        for documento in documentos['DocumentosGenerales']:
            Documentos_generales.append(documento)
        for documento in documentos['DocumentosEconomicos']:
            Documentos_economicos.append(documento)
        for documento in documentos['DocumentosJuridicos']:
            Documentos_juridicos.append(documento)
    except KeyError:
        print(f'Could not find Documentos online for {company["name"]}, ruc: {company["ruc"]}')
        continue

Could not find Documentos online for INGERSA SA, ruc: 0990331014001
Could not find Documentos online for DEPOSITO CENTRALIZADO DE COMPENSACION Y LIQUIDACION DE VALORES DECEVALE S.A., ruc: 0991283765001
Could not find Documentos online for CURTIEMBRE RENACIENTE S.A., ruc: 0190004937001
Could not find Documentos online for MECANICA DE PRECISION LEMA DEL PACIFICO MEPRELPA S.A., ruc: 0190152588001
Could not find Documentos online for PRODUCTOS OLE DEL ECUADOR PRODUCTOSOLE CIA.LTDA., ruc: 1791288246001
Could not find Documentos online for FLOWERCARGO S.A.S., ruc: 1791278410001
Could not find Documentos online for HONTANAR CIA. LTDA., ruc: 1791397398001
Could not find Documentos online for PORMAR TRANSPORTE POR MAR S. A., ruc: 0990872511001
Could not find Documentos online for IMPUBLI CIA. LTDA., ruc: 0992717955001
Could not find Documentos online for CASUR S. A., ruc: 0990856125001
Could not find Documentos online for ARBUSTA C LTDA, ruc: 1790685411001
Could not find Documentos online for A

Get all of the difrent kind of documentos generales into a df

In [24]:
def proces_documents(documentos, label_fields):
    docs = []
    for documento in documentos:
        fields = documento.split('.')[0].split('_')
        new_doc = {}
        for index, field in enumerate(fields):
            new_doc[label_fields[index]] = field
        # add filename at the end
        new_doc['filename'] = documento
        # add new doc
        docs.append(new_doc)
    return docs

doc_generales_fields = ['ruc', 'tipo_doc', 'documento', 'fecha', 'nombre', 'cargo', 'other']
docs = proces_documents(Documentos_generales, doc_generales_fields)
df_generales = pd.DataFrame(docs)
df_generales.head()

,ruc,tipo_doc,documento,fecha,filename,nombre,cargo
0,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,1994-09-13,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN
1,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,1996-09-06,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN
2,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,2008-05-08,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN
3,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,2009-01-14,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN
4,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,2013-05-13,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN


In [38]:
# get all the ditribution of the documents
print(df_generales['documento'].value_counts())

documento
Oficio Nombramiento Administradores                        169283
Formulario de actualización de datos                        88132
Oficio Transferencia Acciones                               84519
Nombramiento                                                43904
Razón de Inscripción registrada                             39916
Formulario de Información de la Sociedad Extranjera         27441
Certificado de Existencia Legal                             27440
Poder otorgado por la Sociedad Extranjera                   26959
Oficio de Presentación de Doc                               15865
Declaración Juramentada  Nómina de Socios o Accionistas     15635
Solicitud de acceso y declaración de responsabilidad        13791
Escritura Cesión Participación                              10221
Carta de autorización                                        7815
Notificación general                                         2730
Renuncia                                                     2355


In [51]:
# get all the documents which have the document 'Oficio Nombramiento Administradores' and have the date containes '2023'
df_oficio_nombramiento_administradores = df_generales[(df_generales['documento'] == 'Oficio Nombramiento Administradores') & (df_generales['fecha'].str.contains('2023'))]

df_oficio_nombramiento_administradores.head()

# for every document we have the filename, so we can download it
import gridfs
from io import BytesIO


,ruc,tipo_doc,documento,fecha,filename,nombre,cargo
104,0991295437001,DocumentosGenerales,Oficio Nombramiento Administradores,2023-10-19,0991295437001_DocumentosGenerales_Oficio Nombr...,NAVARRO HUFENBACH GUSTAVO DANIEL,VICEPRESIDENTE
254,0791842041001,DocumentosGenerales,Oficio Nombramiento Administradores,2023-01-25,0791842041001_DocumentosGenerales_Oficio Nombr...,CARTUCHE ESCALERAS CRISTHIAN HAMILTON,PRESIDENTE
257,0791842041001,DocumentosGenerales,Oficio Nombramiento Administradores,2023-01-25,0791842041001_DocumentosGenerales_Oficio Nombr...,CARTUCHE ESCALERAS ROGER EDUARDO,GERENTE GENERAL
507,1792369428001,DocumentosGenerales,Oficio Nombramiento Administradores,2023-03-17,1792369428001_DocumentosGenerales_Oficio Nombr...,RAMIREZ TOVAR MELISA LOURDES,PRIMER DIRECTOR EJECUTIVO
508,1792369428001,DocumentosGenerales,Oficio Nombramiento Administradores,2023-12-18,1792369428001_DocumentosGenerales_Oficio Nombr...,TRUJILLO LOPEZ JASON LEONARDO,TERCER DIRECTOR EJECUTIVO


In [25]:
docs_economicos_fields = ['ruc', 'tipo_doc', 'documento', 'fecha', 'other']
docs = proces_documents(Documentos_economicos, docs_economicos_fields)
df_economicos = pd.DataFrame(docs)
df_economicos.head()

,ruc,tipo_doc,documento,fecha,filename
0,0991295437001,DocumentosEconomicos,Balance Estado de Situación Financiera,1994-12-31,0991295437001_DocumentosEconomicos_Balance Es...
1,0991295437001,DocumentosEconomicos,Balance Estado de Situación Financiera,1995-12-31,0991295437001_DocumentosEconomicos_Balance Es...
2,0991295437001,DocumentosEconomicos,Balance Estado de Situación Financiera,1996-12-31,0991295437001_DocumentosEconomicos_Balance Es...
3,0991295437001,DocumentosEconomicos,Balance Estado de Situación Financiera,1997-12-31,0991295437001_DocumentosEconomicos_Balance Es...
4,0991295437001,DocumentosEconomicos,Balance Estado de Situación Financiera,1998-12-31,0991295437001_DocumentosEconomicos_Balance Es...


In [45]:
# get all the ditribution of the documents economicosn 
print(df_economicos['documento'].value_counts())

documento
Balance  Estado de Situación Financiera                         292548
Nómina de Socios  Accionistas                                   285830
Informe de Gerente                                              247390
RUC                                                             201284
Nómina de Administradores                                       197495
Estado de Flujos de Efectivo                                    181122
Estado de Cambios en el Patrimonio                              181120
Notas a los Estados Financieros                                 180977
Informe de Comisario                                            178803
Acta de la Junta General                                        178201
Auditoria Externa                                               144411
Estado de Resultado Integral                                     72634
Formulario de Actualización de Datos                             19428
Conciliación del Estado de Resultado Integral                     9

In [26]:
docs_juridicos_fields = ['ruc', 'tipo_doc', 'acto_juridico', 'fecha', 'other']
docs = proces_documents(Documentos_juridicos, docs_juridicos_fields)
df_juridicos = pd.DataFrame(docs)
df_juridicos.head()

,ruc,tipo_doc,acto_juridico,fecha,other,filename
0,0991295437001,DocumentosJuridicos,CONSTITUCIÓN,Escritura,1994-09-01,0991295437001_DocumentosJuridicos_CONSTITUCIÓN...
1,0991295437001,DocumentosJuridicos,CONSTITUCIÓN,Extracto,1994-09-01,0991295437001_DocumentosJuridicos_CONSTITUCIÓN...
2,0991295437001,DocumentosJuridicos,AUMENTO DE CAPITAL,Escritura,2002-12-31,0991295437001_DocumentosJuridicos_AUMENTO DE C...
3,0991295437001,DocumentosJuridicos,AUMENTO DE CAPITAL,Extracto,2002-12-31,0991295437001_DocumentosJuridicos_AUMENTO DE C...
4,0991295437001,DocumentosJuridicos,CAMBIO DE OBJETO SOCIAL,Escritura,2006-02-16,0991295437001_DocumentosJuridicos_CAMBIO DE OB...


In [46]:
# get all the ditribution of the documents economicosn 
print(df_juridicos['acto_juridico'].value_counts())

acto_juridico
AUMENTO DE CAPITAL                                                                                      64973
CONSTITUCIÓN                                                                                            48224
CAMBIO DE OBJETO SOCIAL                                                                                  9293
OTRAS REFORMAS                                                                                           8885
CAMBIO DE DOMICILIO                                                                                      5250
                                                                                                        ...  
PRORROGA O DISMINUCIÓN DE PLAZO, OTRAS REFORMAS, CONVALIDACIÓN                                              1
FIJAR EL CAPITAL AUTORIZADO, REFORMA INTEGRAL DE ESTATUTOS                                                  1
REACTIVACIÓN, REFORMA INTEGRAL DE ESTATUTOS                                                               

Now get all of the the documentos economicos

Check the pdfs for the pdfs that we don't have the complete title or filename for
this it important because based on the document name we define how we can extract the information 

In [31]:
#print(f'In the df_generales there are {df_generales_nan.shape[0]} out of {df_generales.shape[0]} with no fecha column')
#df_generales['documento'].value_counts()

,ruc,tipo_doc,documento,fecha,filename,nombre,cargo
0,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,1994-09-13,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN
1,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,1996-09-06,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN
2,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,2008-05-08,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN
3,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,2009-01-14,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN
4,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,2013-05-13,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN


In [7]:

# make the same for economicos
#df_economicos['documento'].value_counts().shape

In [ ]:
df_juridicos_fecha_nan = df_juridicos[df_juridicos['fecha'].isna()]

df_juridicos_fecha_nan.shape

# get all docuemntos de 2023
#df_juridicos[df_juridicos['fecha'].str.contains('2023')].head()
#df_juridicos['acto_juridico'].value_counts().shape
# make the same for juridicos
#df_juridico_2023 = df_juridicos['acto_juridico'].value_counts()
